# Filtration vs. Sedimentation
## Part 3: Model Comparison


During the Week 4 lecture, we walked through how to construct a custom filtration model, and throughout this post-session tutorial,
we have constructed a simpler custom sedimentation model. Now, we'll compare the model formulations to highlight the key similarities and differences.

```
     ┌─────────────────────┐
     │                     │ → OVERFLOW (clear water)
IN → │  SEDIMENTATION TANK │
     │                     │ ↓ UNDERFLOW (sludge)
     └─────────────────────┘

     ┌─────────────────────┐
     │                     │ → Outlet (filtered water)
IN → │     Filtration      │
     │                     │ ↓ Waste (sludge)
     └─────────────────────┘
```
---
These two models are structurally identical, but have the following key differences:
- Number of components
- Complexity of calculations
- Optional features to improve model robustness

**Goal**: Understand the advantages of different levels of model complexity

**Time**: 10-15 minutes

## Section 1: Property Models
**Both property models follow the same structure:**
1. Define the components
2. Define the parameters.
   
**The only difference is in model complexity.**

### Sedimentation Tank Parameter Block (Simple)
<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
# Two components
self.H2O = Solvent()
self.TSS = Component()

# Assumes a constant density of 1000 kg/m3
self.dens_mass = Param(initialize=1000, units=pyunits.kg/pyunits.m**3)

### Filtration Parameter Block (Complex)
<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
# Three components
self.H2O = Solvent()
self.TSS = Component()
self.NaCl = Component()

# More complex density calculation: ρ = 756*C + 995, where C is the mass fraction of sodium chloride. Here, we define the parameter values (`dens_mass_param_dict`), 756 and 995 for the linear density relationship.
dens_mass_param_dict = {"0": 995, "1": 756}
self.dens_mass_param = Param(
    dens_mass_param_dict.keys(),
    domain=Reals,
    initialize=dens_mass_param_dict,
    mutable=True,
    units=pyunits.kg / pyunits.m**3,
)

# Density constraint - this is where we define the equation as ρ = 756*C + 995
def rule_dens_mass_phase(b):
    return (
        b.dens_mass_phase["Liq"]
        == b.params.dens_mass_param["1"] * b.mass_frac_phase_comp["Liq", "NaCl"]
        + b.params.dens_mass_param["0"]
    )
self.eq_dens_mass_phase = Constraint(rule=rule_dens_mass_phase)

## Section 2: Unit Models
**Both unit models follow the same structure:**
1. Define the unit variables
2. Create the state blocks
3. Create the relevant constraints.

### Sedimentation Unit Model (Simple)
<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
# Unit variables
self.water_recovery = Var(initialize=0.99, bounds=(0, 1))
self.solids_removal = Var(initialize=0.70, bounds=(0, 1))

# State blocks
self.properties_in = ...
self.properties_overflow = ...
self.properties_underflow = ...

# Constraints
@self.Constraint(...)
def mass_balance(b, j):
    return (
        b.properties_in[0].flow_mass_phase_comp["Liq", j]
        == b.properties_overflow[0].flow_mass_phase_comp["Liq", j]
        + b.properties_underflow[0].flow_mass_phase_comp["Liq", j]
    )

@self.Constraint(...)
def water_recovery_eq(b):
    return (
        b.properties_overflow[0].flow_mass_phase_comp["Liq", "H2O"]
        == b.water_recovery * b.properties_in[0].flow_mass_phase_comp["Liq", "H2O"]
    )

@self.Constraint(...)
def solids_removal_eq(b):
    return (
        b.properties_underflow[0].flow_mass_phase_comp["Liq", "TSS"]
        == b.solids_removal * b.properties_in[0].flow_mass_phase_comp["Liq", "TSS"]
    )

### Filtration Unit Model (Complex)
<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
# Unit variables - same pattern, but includes indexing
self.recovery_mass_phase_comp = Var(
    self.config.property_package.phase_list,
    self.config.property_package.component_list,
    initialize=0.5,
    bounds=(1e-8, 1),
    units=pyunits.dimensionless,
)
self.removal_fraction_mass_phase_comp = Var(
    self.config.property_package.phase_list,
    self.config.property_package.component_list,
    initialize=0.5,
    bounds=(1e-8, 1),
    units=pyunits.dimensionless,
)

# State blocks - same pattern as Sedimentation
self.properties_in = ...
self.properties_out = ...
self.properties_waste = ...

# Constraints - same pattern, but includes indexing for recovery and removal constraints
@self.Constraint(self.config.property_package.component_list)
def eq_mass_balance(b, j):
    return (
        b.properties_in[0].flow_mass_phase_comp["Liq", j]
        == b.properties_out[0].flow_mass_phase_comp["Liq", j]
        + b.properties_waste[0].flow_mass_phase_comp["Liq", j]
    )

@self.Constraint(self.config.property_package.component_list)
def eq_recovery_balance(b, j):
    return (
        b.removal_fraction_mass_phase_comp["Liq", j]
        == 1 - b.recovery_mass_phase_comp["Liq", j]
    )

@self.Constraint(self.config.property_package.component_list)
def eq_removal_balance(b, j):
    return (
        b.properties_in[0].flow_mass_phase_comp["Liq", j]
        * b.removal_fraction_mass_phase_comp["Liq", j]
        == b.properties_waste[0].flow_mass_phase_comp["Liq", j]
    )
```

The sedimentation and filtration models both have 2 variables: recovery and removal fraction. In the sedimentation unit model, the recovery variable is explicitly defined as the recovery of water, and the removal fraction is explicitly defined as the removal fraction of TSS. However, in the filtration unit model, both variables are indexed by phase and component. Likewise, the constraints in the filtration model calculate the recovery and removal for each component (H2O, TSS, and NaCl), whereas the constraints in the sedimentation model only calculate the recovery of water and removal of TSS. **Thus, the main difference between these two models is that filtration uses the component list index to loop through all the components, while sedimentation explicitly writes out all of its components.**

## Section 3: Additional Features

Now let's go over the optional add-ons that the Filtration model included (and the Sedimentation model excluded).


### Filtration State Block Initialization Routine (Complex)

This initialization routine helps the solver find a solution more readily, but the Sedimentation model does not require this since the model is relatively simple.

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
class _CustomStateBlock(StateBlock):
    def initialize(self, ...):
    # Step-by-step startup procedure
        

### Filtration State Block Scaling Routine (Complex)

Similarly, the scaling routine is critical for numerical stability in complex systems and helps the solver find a solution more readily, but the Sedimentation model does not require the same level of model robustness.

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
@declare_process_block_class("CustomStateBlock", block_class=_CustomStateBlock)
class CustomStateBlockData(StateBlockData):
    def calculate_scaling_factors(self):
        super().calculate_scaling_factors()

### Unit Model Configuration Options

While both models have the same configuration options, the Sedimentation model removes the optional "description" and "doc" arguments.

## Sedimentation
<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
    # Configuration options
    CONFIG = ConfigBlock()
    CONFIG.declare(
        "dynamic",
        ConfigValue(default=False, domain=In([False])),
    )
    CONFIG.declare(
        "has_holdup",
        ConfigValue(default=False, domain=In([False])),
    )
```

## Filtration
<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
    # Configuration options
    CONFIG = ConfigBlock()

    CONFIG.declare(
        "dynamic",
        ConfigValue(
            domain=In([False]),
            default=False,
            description=...,
            doc=...,
        ),
    )
    CONFIG.declare(
        "has_holdup",
        ConfigValue(
            default=False,
            domain=In([False]),
            description=...,
            doc=...,
        ),
    )

## Summary

Now that we've compared aspects of the Filtration and Sedimentation models side-by-side, you can see that the models share the same structure, just with different levels of complexity:

```
1. Define components and parameters (Parameter Block - Property Model)
   ↓
2. Create state variables (State Block - Property Model)
   ↓
3. Add unit variables (Unit Model)
   ↓
4. Write constraints (Unit Model)
   ↓
5. Solve!
```

**Key Takeaway:** The point of Week 4's lecture and these post-session tutorials is NOT to have you memorize Pyomo syntax or understand every intricacy of modeling concepts like scaling or initialization. The main goal is for you to familiarize yourself with the structure of WaterTAP models such that you can utilize existing code, modify it to suit your particular application, and/or develop custom models for your own system. 